In [29]:
import sys
sys.path.append("..")
from utils.azure_utils import KeyVault, DataLake
from utils.data_pre_utils import df_val_map
import numpy as np
import pandas as pd

In [30]:
# Connect to Storage Account
vault = KeyVault(keyVaultName = "keyvaultdva2022")
storage_credential = vault.get_secret(secretName = "storagePrimaryKey")
storage = DataLake(account_name = "storageaccountdva", credential = storage_credential)

In [31]:
file_system = "energyhub"
directory_raw = "/data_original/meters/cleaned"
directory_clean = "/data/meters"

In [32]:
paths = storage.list_directory_contents(file_system, directory_raw, print_paths=False)
file_names = []
for file in paths:
    file_names.append(file.name.split("/")[3])

In [33]:
file_names

['chilledwater_cleaned.csv',
 'electricity_cleaned.csv',
 'gas_cleaned.csv',
 'hotwater_cleaned.csv',
 'irrigation_cleaned.csv',
 'solar_cleaned.csv',
 'steam_cleaned.csv',
 'water_cleaned.csv']

In [34]:
for name in file_names:
    df = storage.read(file_system, directory_raw, file_name=name)
    df_tf = df.melt(id_vars='timestamp')
    assert len(df) *(len(df.columns)-1) == len(df_tf)
    new_name = name.split("_")[0]
    storage.write(file_system, directory_clean, file = df_tf, file_name = new_name, extension="csv", overwrite = True)

    